In [1]:
pip install --upgrade google-cloud-videointelligence

Requirement already up-to-date: google-cloud-videointelligence in /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages (1.15.0)
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
# Update this to point to your IAM json file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../Creds/cloudaiguild-88fff7598286.json"

In [3]:
%%bash 
gcloud services enable videointelligence.googleapis.com

In [4]:
from google.cloud import videointelligence

video_client = videointelligence.VideoIntelligenceServiceClient()
features = [videointelligence.enums.Feature.LABEL_DETECTION]
operation = video_client.annotate_video(
    'gs://cloud-samples-data/video/cat.mp4', features=features)
print('\nProcessing video for label annotations:')

result = operation.result(timeout=120)
print('\nFinished processing.')

# first result is retrieved because a single video was processed
segment_labels = result.annotation_results[0].segment_label_annotations
for i, segment_label in enumerate(segment_labels):
    print('Video label description: {}'.format(
        segment_label.entity.description))
    for category_entity in segment_label.category_entities:
        print('\tLabel category description: {}'.format(
            category_entity.description))

    for i, segment in enumerate(segment_label.segments):
        start_time = (segment.segment.start_time_offset.seconds +
                      segment.segment.start_time_offset.nanos / 1e9)
        end_time = (segment.segment.end_time_offset.seconds +
                    segment.segment.end_time_offset.nanos / 1e9)
        positions = '{}s to {}s'.format(start_time, end_time)
        confidence = segment.confidence
        print('\tSegment {}: {}'.format(i, positions))
        print('\tConfidence: {}'.format(confidence))
    print('\n')


Processing video for label annotations:

Finished processing.
Video label description: tabby cat
	Label category description: cat
	Segment 0: 0.0s to 14.84s
	Confidence: 0.30844953656196594


Video label description: maine coon
	Label category description: cat
	Segment 0: 0.0s to 14.84s
	Confidence: 0.32171761989593506


Video label description: pet
	Label category description: animal
	Segment 0: 0.0s to 14.84s
	Confidence: 0.8389395475387573


Video label description: cat
	Label category description: pet
	Segment 0: 0.0s to 14.84s
	Confidence: 0.997473955154419


Video label description: small to medium sized cats
	Label category description: mammal
	Segment 0: 0.0s to 14.84s
	Confidence: 0.7987512946128845


Video label description: kitten
	Label category description: cat
	Segment 0: 0.0s to 14.84s
	Confidence: 0.3615228831768036


Video label description: animal
	Segment 0: 0.0s to 14.84s
	Confidence: 0.9441419243812561


Video label description: whiskers
	Segment 0: 0.0s to 14.84s

In [5]:
%%bash
gsutil cp 'gs://cloud-samples-data/video/cat.mp4' cat.mp4

Copying gs://cloud-samples-data/video/cat.mp4...
- [1 files][  6.1 MiB/  6.1 MiB]                                                
Operation completed over 1 objects/6.1 MiB.                                      


In [6]:
%%HTML
<video width="320" height="240" controls>
  <source src="cat.mp4" type="video/mp4">
</video>

In [7]:
%%bash

generatePostData()
{
    cat <<EOF
{
    "inputUri":"gs://cloud-samples-data/video/cat.mp4",
    "features": [
       "LABEL_DETECTION"
    ]
}
EOF
}

curl -X POST \
  -H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
  -H "Content-Type: application/json; charset=utf-8" \
     --data "$(generatePostData)" "https://videointelligence.googleapis.com/v1/videos:annotate"


{
  "name": "projects/640034225016/locations/us-east1/operations/5018424613092155472"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   196    0    88  100   108    142    174 --:--:-- --:--:-- --:--:--   316


In [9]:
%%bash

curl -X GET \
   -H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
   https://videointelligence.googleapis.com/v1/projects/640034225016/locations/us-east1/operations/3485700468051400564

{
  "name": "projects/640034225016/locations/us-east1/operations/3485700468051400564",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.videointelligence.v1.AnnotateVideoProgress",
    "annotationProgress": [
      {
        "inputUri": "/cloud-samples-data/video/cat.mp4",
        "progressPercent": 100,
        "startTime": "2020-09-15T19:31:39.713487Z",
        "updateTime": "2020-09-15T19:32:01.808466Z"
      }
    ]
  },
  "done": true,
  "response": {
    "@type": "type.googleapis.com/google.cloud.videointelligence.v1.AnnotateVideoResponse",
    "annotationResults": [
      {
        "inputUri": "/cloud-samples-data/video/cat.mp4",
        "segmentLabelAnnotations": [
          {
            "entity": {
              "entityId": "/m/0g4cd0",
              "description": "tabby cat",
              "languageCode": "en-US"
            },
            "categoryEntities": [
              {
                "entityId": "/m/01yrx",
                "description": "cat",
      

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9533    0  9533    0     0  19737      0 --:--:-- --:--:-- --:--:-- 19696
